##### Check the subjects of the clips and extract the key frames of each 

In [ ]:
import pandas as pd
import cv2 
import numpy as np
import os
import moviepy
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip
from timeit import default_timer as timer
import operator
import matplotlib.pyplot as plt
import sys
from tqdm import tqdm
from scipy.signal import argrelextrema
from joblib import Parallel, delayed


###### clips info 

In [ ]:
clip_dir = "../data/USF/clips/" # your clip folders
clip_dir_nopain = clip_dir + "no_pain/"
clip_dir_pain = clip_dir + "pain/"
print(os.path.isdir(clip_dir_nopain), os.path.isdir(clip_dir_pain))

In [ ]:
# obtain all the files
no_pain_files =[clip_dir_nopain + i for i in os.listdir(clip_dir_nopain)]
pain_files = [clip_dir_pain + i for i in os.listdir(clip_dir_pain)]
print("number of no pain files:{}, and that of pain files:{}".format(len(no_pain_files), len(pain_files)))

###### Get the  number of subjects

In [ ]:
no_pain_subjects =[]
pain_subjects = []
for item in no_pain_files:
    no_pain_subjects.append(item.split('_')[-2])
print("num of no pain subjects: {}".format(len(set(no_pain_subjects))))

In [ ]:
for item in pain_files:
    pain_subjects.append(item.split('_')[-2])
print("num of pain subjects: {}".format(len(set(pain_subjects))))

In [ ]:
no_pain_subjects = set(no_pain_subjects)
pain_subjects = set(pain_subjects)
print(no_pain_subjects, pain_subjects)

###### check the clip files  

In [ ]:
for item in no_pain_files:
    if not os.path.isfile(item):
        print("the file does exist:{}".format(item))
for item in pain_files:
    if not os.path.isfile(item):
        print("the file does exist:{}".format(item))

In [ ]:
print("number of no pain files:{}, and that of pain files:{}".format(len(no_pain_files), len(pain_files)))

###### Extract the frames based on local maxima

In [ ]:
def smooth(x, window_len=13, window='hanning'):
    """smooth the data using a window with requested size.
    
    This method is based on the convolution of a scaled window with the signal.
    The signal is prepared by introducing reflected copies of the signal 
    (with the window size) in both ends so that transient parts are minimized
    in the begining and end part of the output signal.
    
    input:
        x: the input signal 
        window_len: the dimension of the smoothing window
        window: the type of window from 'flat', 'hanning', 'hamming', 'bartlett', 'blackman'
            flat window will produce a moving average smoothing.
    output:
        the smoothed signal
        
    example:
    import numpy as np    
    t = np.linspace(-2,2,0.1)
    x = np.sin(t)+np.random.randn(len(t))*0.1
    y = smooth(x)
    
    see also: 
    
    numpy.hanning, numpy.hamming, numpy.bartlett, numpy.blackman, numpy.convolve
    scipy.signal.lfilter
 
    TODO: the window parameter could be the window itself if an array instead of a string   
    """
#     print(len(x), window_len)
#     if x.ndim != 1:
#         raise ValueError, "smooth only accepts 1 dimension arrays."

#     if x.size < window_len:
#         raise ValueError, "Input vector needs to be bigger than window size."

#     if window_len < 3:
#         return x

#     if not window in ['flat', 'hanning', 'hamming', 'bartlett', 'blackman']:
#         raise ValueError, "Window is on of 'flat', 'hanning', 'hamming', 'bartlett', 'blackman'"

    s = np.r_[2 * x[0] - x[window_len:1:-1],
              x, 2 * x[-1] - x[-1:-window_len:-1]]
    #print(len(s))

    if window == 'flat':  # moving average
        w = np.ones(window_len, 'd')
    else:
        w = getattr(np, window)(window_len)
    y = np.convolve(w / w.sum(), s, mode='same')
    return y[window_len - 1:-window_len + 1]

#Class to hold information about each frame

class Frame:
    def __init__(self, id, frame, value):
        self.id = id
        self.frame = frame
        self.value = value

    def __lt__(self, other):
        if self.id == other.id:
            return self.id < other.id
        return self.id < other.id

    def __gt__(self, other):
        return other.__lt__(self)

    def __eq__(self, other):
        return self.id == other.id and self.id == other.id

    def __ne__(self, other):
        return not self.__eq__(other)

def rel_change(a, b):
    x = (b - a) / max(a, b)
    print(x)
    return x

In [ ]:
print(sys.executable)
#Setting fixed threshold criteria
USE_THRESH = False
#fixed threshold value
THRESH = 0.6
#Setting fixed threshold criteria
USE_TOP_ORDER = False
#Setting local maxima criteria
USE_LOCAL_MAXIMA = True
#Number of top sorted frames
NUM_TOP_FRAMES = 20
len_window = 80
#Video path of the source file
# videopath = video_path
#Directory to store the processed frames
dir = "../data/USF/temple2/"
#smoothing window size


In [ ]:
def capture_keyframes(videopath, output_dir):

#     print("Video :" + videopath)
#     print("Frame Directory: " + output_dir)
    video_name = videopath.split('/')[-1].split('.')[0]

    cap = cv2.VideoCapture(str(videopath))


    curr_frame = None
    prev_frame = None

    frame_diffs = []
    frames = []
    ret, frame = cap.read()
    i = 1

    while(ret):
        luv = cv2.cvtColor(frame, cv2.COLOR_BGR2LUV)
        curr_frame = luv
        if curr_frame is not None and prev_frame is not None:
            #logic here
            diff = cv2.absdiff(curr_frame, prev_frame)
            count = np.sum(diff)
            frame_diffs.append(count)
            frame = Frame(i, frame, count)
            frames.append(frame)
        prev_frame = curr_frame
        i = i + 1
        ret, frame = cap.read()
    """
        cv2.imshow('frame',luv)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    """
    cap.release()
    #cv2.destroyAllWindows()

    if USE_TOP_ORDER:
        # sort the list in descending order
        frames.sort(key=operator.attrgetter("value"), reverse=True)
        for keyframe in frames[:NUM_TOP_FRAMES]:
            name = "frame_" + str(keyframe.id) + ".jpg"
            cv2.imwrite(output_dir + "/" + name, keyframe.frame)

    if USE_THRESH:
        print("Using Threshold")
        for i in range(1, len(frames)):
            if (rel_change(np.float(frames[i - 1].value), np.float(frames[i].value)) >= THRESH):
                #print("prev_frame:"+str(frames[i-1].value)+"  curr_frame:"+str(frames[i].value))
                name = "frame_" + str(frames[i].id) + ".jpg"
                cv2.imwrite(output_dir + "/" + name, frames[i].frame)


    if USE_LOCAL_MAXIMA:
#         print("Using Local Maxima")
        diff_array = np.array(frame_diffs)
        sm_diff_array = smooth(diff_array, len_window)
        frame_indexes = np.asarray(argrelextrema(sm_diff_array, np.greater))[0]
        if "sub053_pro1" in item:
            print("find sub53")
            for i in frame_indexes:
                name = "frame_" + str(frames[i - 1].id) + video_name + ".jpg"
                #print(dir+name)
                cv2.imwrite(output_dir + name, cv2.rotate(frames[i - 1].frame,cv2.ROTATE_90_COUNTERCLOCKWISE))
        else:
#             print("not find sub53")
            for i in frame_indexes:
                name = "frame_" + str(frames[i - 1].id) + video_name + ".jpg"
                #print(dir+name)
                cv2.imwrite(output_dir + name, cv2.rotate(frames[i - 1].frame,cv2.ROTATE_90_CLOCKWISE))

#     plt.figure(figsize=(40, 20))
#     plt.locator_params(numticks=100)
#     plt.stem(sm_diff_array)
#     plt.savefig(dir + 'plot.png')


In [ ]:
#create keyframe folder
%mkdir "../data/USF/keyframe/"
%mkdir "../data/USF/keyframe/pain"
%mkdir "../data/USF/keyframe/no_pain"

In [ ]:
no_pain_frames = "../data/USF/keyframe/no_pain/"
pain_frames = "../data/USF/keyframe/pain/"

In [ ]:
for item in tqdm(pain_files):
    capture_keyframes(item, pain_frames)

In [ ]:
for item in tqdm(no_pain_files):
    capture_keyframes(item, no_pain_frames)